# Pipeline de ELT para Dados de Vendas

Para simular uma base de dados de um e-comerce, utilizarei o site kaggle.com.
Abaixo segue o link para o arquivo utilizado:
https://www.kaggle.com/datasets/carrie1/ecommerce-data

In [1]:
# Importando bibliotecas para manipulação do arquivo:
import zipfile as zp
import os
import pandas as pd
import numpy as np
from google.cloud import bigquery

# Mapeando o diretório de origem e destino da extração do arquivo .zip

In [2]:
origem_zip = 'D:/Estudos/Projetos Pessoais/elt_ecommerce/dataset/archive.zip'
destino_zip = 'D:/Estudos/Projetos Pessoais/elt_ecommerce/dataset/'
dataset = 'D:/Estudos/Projetos Pessoais/elt_ecommerce/dataset/data.csv'
global dataset

In [3]:
# Criando o diretório caso não exista ainda:
os.makedirs(destino_zip, exist_ok=True)

# Extraindo o arquivo:

In [4]:
with zp.ZipFile(origem_zip, 'r') as zip_ref:
    zip_ref.extractall(destino_zip)

print(f'Arquivos extraídos para {destino_zip}')

Arquivos extraídos para D:/Estudos/Projetos Pessoais/elt_ecommerce/dataset/


# Identificando a encodificação do arquivo:

In [5]:
import chardet

# Function to detect encoding of a file
def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        # Read a portion of the file, as full file reading might be unnecessary and slow
        raw_data = file.read(100000)  # Read the first 100,000 bytes
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        confidence = result['confidence']
    return encoding, confidence

# Path to your file
file_path = dataset

# Detect encoding
encoding, confidence = detect_encoding(file_path)
print(f"Detected encoding: {encoding} with confidence: {confidence}")

Detected encoding: ascii with confidence: 1.0


# Lendo o arquivo:

In [6]:
df = pd.read_csv(dataset, encoding='ISO-8859-1', dtype={'CustomerID': str})
print(df.head(10))

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
5    536365     22752         SET 7 BABUSHKA NESTING BOXES         2   
6    536365     21730    GLASS STAR FROSTED T-LIGHT HOLDER         6   
7    536366     22633               HAND WARMER UNION JACK         6   
8    536366     22632            HAND WARMER RED POLKA DOT         6   
9    536367     84879        ASSORTED COLOUR BIRD ORNAMENT        32   

      InvoiceDate  UnitPrice CustomerID         Country  
0  12/1/2010 8:26       2.55      17850  United Kingdom  
1  12/1/2010 8:26       3.39      17850  United Kingdom  
2  12/1/2010 8:26       2.75     

# Credenciais google
Definindo o caminho para a chave .json usada na API de conexão

In [7]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "D:/Estudos/Projetos Pessoais/elt_ecommerce/acces_key_gcp/key.json"

# Limpando a tabela caso exista

In [8]:
def tabela_existe(client, dataset_id, table_id):
    try:
        client.get_table(f'{dataset_id}.{table_id}')
        return True
    except Exception as e:
        if 'Not found' in str(e):
            return False
        else:
            raise

def truncar_tabela(client, dataset_id, table_id):
    query = f'TRUNCATE TABLE `{dataset_id}.{table_id}`'
    query_job = client.query(query)
    query_job.result() # Aguarda o fim da execução
    print(f' A tabela {dataset_id}.{table_id} foi truncada')

def main():
    client = bigquery.Client()
    dataset_id = 'ecomerce'
    table_id = 'ecomerce'

    if tabela_existe(client, dataset_id, table_id):
        truncar_tabela(client, dataset_id, table_id)
    else:
        print(f'A tabela {dataset_id}.{table_id} não existe')

if __name__ == '__main__':
    main()

 A tabela ecomerce.ecomerce foi truncada


# Carregando os dados coletados para o GCP

In [9]:
#Instalada a biblioteca google cloud bigquery
#pip install google-cloud-bigquery

In [10]:
from google.cloud import bigquery

# Parâmetros
dataset_id = 'ecomerce'
table_id = 'ecomerce'
source_file_name = 'D:/Estudos/Projetos Pessoais/elt_ecommerce/dataset/data.csv'

def carga_gbq(dataset_id, table_id, source_file_name):
    #Cria um cliente BigQuery
    client = bigquery.Client()

    #Define o dataset e a tabela
    table_ref = client.dataset(dataset_id).table(table_id)

    #Configura o job de carregamento
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1, #Pular a primeira linha(cabeçalho)
        autodetect=True,     #Tentar detectar o esquema automáticamente(Colunas)
    )

    #Lê o arquivo CSV e inicia o job de carregamento
    with open(source_file_name, "rb") as source_file:
        load_job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

    print('Iniciando o job de carregamento {}'.format(load_job.job_id))

    load_job.result() #Espera o job ser concluído

    print('Carregamento concluído.')

    #Verifica o número de linhas carregadas
    destination_table = client.get_table(table_ref)
    print('Carregou {} linhas.'.format(destination_table.num_rows))




In [11]:
#Executa a carga
carga_gbq(dataset_id, table_id, source_file_name)

Iniciando o job de carregamento 7a90878b-f29b-44e8-9275-9372cbaaf5a6
Carregamento concluído.
Carregou 541909 linhas.


# Executando uma consulta de testes

In [ ]:
client = bigquery.Client()
query = "select distinct * from `probable-tape-423914-a4.ecomerce.ecomerce` where description is not null and description != '?'"

query_job = client.query(query)

#Resultados 
query_result = query_job.result()

for row in query_result:
    print(f'Description: {row.Description}')
